In [19]:
import pandas, copy
from gumpy.genome import Genome

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:

TEST_CASE_DIR = "tests/test-cases/"

In [21]:
reference=Genome(genbank_file="config/NC_004148.2.gbk",name="HMPV")

In [22]:
sample_01=copy.deepcopy(reference)
sample_01.apply_vcf_file(vcf_file=TEST_CASE_DIR+"01.vcf",ignore_status=True,ignore_filter=True,metadata_fields=['GT_CONF','GT_CONF_PERCENTILE'])


In [23]:
sample_01.list_variants_wrt(reference)

['4687t>c', '4725t>c', '4735_indel']

In [28]:
sample_01.table_variants_wrt(reference)

-37 -37
2 1
      VARIANT REF  ALT  GENOME_INDEX GENE  POSITION  NUCLEOTIDE_NUMBER  \
0     4687t>c   t    c          4687   M2       -37                -37   
1     4725t>c   t    c          4725   M2         1                  2   
2  4735_indel   t  ccc          4735   M2        12                 12   

   AMINO_ACID_NUMBER  IS_SNP  IS_INDEL  INDEL_LENGTH ELEMENT_TYPE  
0                NaN    True     False           NaN         GENE  
1                1.0    True     False           NaN         GENE  
2                4.0   False      True           2.0         GENE  


,VARIANT,REF,ALT,GENOME_INDEX,GENE,POSITION,NUCLEOTIDE_NUMBER,AMINO_ACID_NUMBER,IS_SNP,IS_INDEL,INDEL_LENGTH,ELEMENT_TYPE
0,4687t>c,t,c,4687,M2,-37,-37,NaN,True,False,NaN,GENE
1,4725t>c,t,c,4725,M2,1,2,1.0,True,False,NaN,GENE
2,4735_indel,t,ccc,4735,M2,12,12,4.0,False,True,2.0,GENE


In [33]:
m2=reference.genes['M2']

In [37]:
m2.index[:20]

array([4687, 4688, 4689, 4690, 4691, 4692, 4693, 4694, 4695, 4696, 4697,
       4698, 4699, 4700, 4701, 4702, 4703, 4704, 4705, 4706])

In [38]:
m2.sequence[:20]

array(['t', 't', 'a', 'a', 't', 't', 'a', 'a', 'a', 'a', 'a', 't', 'a',
       'a', 'a', 'a', 't', 'a', 'a', 'a'], dtype='<U1')

In [39]:
len(m2.sequence)

548

In [40]:
len(m2.index)

548

In [41]:
m2.cds_index_start

4724

In [42]:
m2.codons

array(['atg', 'tct', 'cgc', 'aag', 'gct', 'cca', 'tgc', 'aaa', 'tat',
       'gaa', 'gtg', 'cgg', 'ggc', 'aaa', 'tgc', 'aac', 'aga', 'gga',
       'agt', 'gag', 'tgt', 'aag', 'ttt', 'aac', 'cac', 'aat', 'tac',
       'tgg', 'agt', 'tgg', 'cca', 'gat', 'aga', 'tac', 'tta', 'tta',
       'ata', 'aga', 'tca', 'aac', 'tat', 'cta', 'tta', 'aat', 'cag',
       'ctt', 'tta', 'agg', 'aac', 'act', 'gat', 'aga', 'gct', 'gat',
       'ggc', 'cta', 'tca', 'ata', 'ata', 'tca', 'ggc', 'gca', 'ggc',
       'aga', 'gaa', 'gac', 'aga', 'acg', 'caa', 'gat', 'ttt', 'gtt',
       'cta', 'ggt', 'tcc', 'acc', 'aat', 'gtg', 'gtt', 'caa', 'ggt',
       'tat', 'att', 'gat', 'gat', 'aac', 'caa', 'agc', 'ata', 'aca',
       'aaa', 'gct', 'gca', 'gcc', 'tgc', 'tac', 'agt', 'cta', 'cac',
       'aac', 'ata', 'atc', 'aag', 'caa', 'cta', 'caa', 'gaa', 'gtt',
       'gaa', 'gtt', 'agg', 'cag', 'gct', 'aga', 'gat', 'agc', 'aaa',
       'cta', 'tct', 'gac', 'agc', 'aag', 'cat', 'gtg', 'gca', 'ctc',
       'cat', 'aac',